
---

**Based on:**
* [Getting Started with TensorFlow Lite for Microcontrollers on i.MX RT](https://community.nxp.com/t5/eIQ-Machine-Learning-Software/Getting-Started-with-TensorFlow-Lite-for-Microcontrollers-on-i/ta-p/1124103) (Revision 10, for reference)
* [Image classification with TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/tutorials/model_maker_image_classification)

The goal of this notebook is to retrain the MobileNetV1 to classify 5 types of flowers: daisy, dandelion, roses, sunflowers, tulips.

After retraining, model is exported as TensorFlow Lite model (*.tflite*) and next converted to C array (*.h*) that can be imported into an embedded project. Embedded project is based on example coming from **SDK 2.10.1** for i.MXRT1170 named *tensorflow_lite_micro_label_image*.

At the end of the notebook, there will be pop-ups asking you where you want to save created files. They will be available also in */content* folder.

> **Additional info**
> 
> Tested with:
> * Python 3.7.12
> * TensorFlow 2.6.0
> * Model Maker 0.2.4

---



---

### Sanity check

---


In [1]:
!python --version
!pip show tensorflow

Python 3.7.12
Name: tensorflow
Version: 2.6.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: clang, flatbuffers, six, numpy, h5py, tensorboard, opt-einsum, grpcio, wrapt, google-pasta, keras-preprocessing, protobuf, wheel, tensorflow-estimator, absl-py, astunparse, gast, typing-extensions, keras, termcolor
Required-by: kapre



---

### Download dataset
After downloading, by default it should be available in */content* folder  

---


In [2]:
!wget -q http://download.tensorflow.org/example_images/flower_photos.tgz
!tar -xzf "/content/flower_photos.tgz"
print('Dataset downloaded and unpacked')

Dataset downloaded and unpacked



---

### Install TensorFlow Lite Model Maker
TensorFlow Lite Model Maker library is required to be **0.2.4**, otherwise seems to be some API changes in never version (refers to **0.3.2**) and below code is not fully operational.

---


In [3]:
!pip install -q tflite-model-maker==0.2.4
!pip show tflite-model-maker # check version installed

     |████████████████████████████████| 190 kB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 55.6 MB/s 
     |████████████████████████████████| 840 kB 23.5 MB/s 
     |████████████████████████████████| 87 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 65.4 MB/s 
     |████████████████████████████████| 1.2 MB 45.0 MB/s 
     |████████████████████████████████| 636 kB 34.7 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 1.0 MB 21.1 MB/s 
     |████████████████████████████████| 37.1 MB 51 kB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |████████████████████████████████| 352 kB 60.1 MB/s 
     |████████████████████████████████| 211 kB 72.9 MB/s 
     |████████████████████████████████| 1.1 MB 75.3 MB/s 
     |████████████████████████████████| 200 kB 52.8 MB/s 
Name: tflite-model-maker
Version: 0.2.4
Summary: TFLite Model Maker: a model customization library for on-device applications.
Home-page: 


---

### Prepare dataset and retrain the model
This is a copy of the [*flower_lab.py*](https://community.nxp.com/t5/eIQ-Machine-Learning-Software/Getting-Started-with-TensorFlow-Lite-for-Microcontrollers-on-i/ta-p/1124103?attachment-id=112680) available from [Getting Started with TensorFlow Lite for Microcontrollers on i.MX RT](https://community.nxp.com/t5/eIQ-Machine-Learning-Software/Getting-Started-with-TensorFlow-Lite-for-Microcontrollers-on-i/ta-p/1124103)

---


In [4]:
import os
from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader
from tflite_model_maker import model_spec

#Specify image directory
image_path = os.path.join(os.getcwd(), 'flower_photos')

#Split up images into different training categories for training, validation, and testing. 
data = ImageClassifierDataLoader.from_folder(image_path)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

#Retrain model on new images
mobilenetv1_spec = model_spec.ImageModelSpec(uri='https://tfhub.dev/google/imagenet/mobilenet_v1_025_128/feature_vector/4')
mobilenetv1_spec.input_image_shape = [128, 128]
model = image_classifier.create(train_data, model_spec=mobilenetv1_spec, validation_data=validation_data)
model.summary()

#Evaluate final model
print('Done training\n')
loss, accuracy = model.evaluate(test_data)

#Write out .tflite file
print('Write out model\n')
model.export(export_dir='.',tflite_filename='flower_model.tflite',label_filename='flower_labels.txt',with_metadata=False)

INFO:tensorflow:Load image with size: 3670, num_label: 5, labels: daisy, dandelion, roses, sunflowers, tulips.
INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 256)               218544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 1285      
Total params: 219,829
Trainable params: 1,285
Non-trainable params: 218,544
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5
91/91 [==============================] - 27s 267ms/step - loss: 1.2459 - accuracy: 0.6446 - val_loss: 0.8197 - val_accuracy: 0.8125
Epoch 2/5
91/91 [==============================] - 24s 261ms/step - loss: 0.8876 - accuracy: 0.7768 - val_loss: 0.7901 - val_accuracy: 0.8153
Epoch 3/5
91/91 [==============================] - 24s 262ms/step - loss: 0.8290 - accuracy: 0.7995 - val_loss: 0.7419 - val_accuracy: 0.8494
Epoch 4/5
91/91 [==============================] - 24s 271ms/step - loss: 0.8284 - accuracy: 0.7988 - val_loss: 0.7553 - val_accuracy: 0.8352
Epoch 5/5
91/91 [==============================] - 27s 296ms/step - loss: 0.8088 - accuracy: 0.8101 - val_loss: 0.7375 - val_accuracy: 0.8608
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 256)               218544    
_________________________________________________________________
dropou

INFO:tensorflow:Assets written to: /tmp/tmpqe5ippro/assets


INFO:tensorflow:Saving labels in ./flower_labels.txt.


INFO:tensorflow:Saving labels in ./flower_labels.txt.



---

### Convert *.tflite* to *.h*

*xxd* is part of [*Vim*](https://www.vim.org/), and provides a method to dump a binary file (in this case *.tflite*) to hex (C array that can be imported into an embedded project)

---


In [9]:
# !apt-get update # in case of problems
!apt-get -qq install xxd
!xxd --version

xxd V1.10 27oct98 by Juergen Weigert


In [10]:
!xxd -i flower_model.tflite > flower_model.h


---

### Download converted model and use with MCUXpresso
3 files will be downloaded:
* retrained model *flower_model.tflite*
* converted model *flower_model.h*
* labels *flowers_labels.txt*

---


In [11]:
from google.colab import files
files.download('flower_model.tflite')
files.download('flower_model.h')
files.download('flower_labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>